In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

# The forward and backward passes

In [2]:
# export
from exp.nb_01 import *

def get_data():
    path = datasets.download_data(MNIST_URL, ext='.gz')
    with gzip.open(path, 'rb') as f:
        ((x_train, y_train), (x_valid, y_valid), _) = pickle.load(f, encoding='latin-1')
    return map(tensor, (x_train,y_train,x_valid,y_valid))
    

In [3]:
# export
def normalize(x,m,s): return (x-m)/s

In [4]:
x_train,y_train,x_valid,y_valid = get_data()

In [5]:
train_mean, train_std = x_train.mean(), x_train.std()
train_mean, train_std

(tensor(0.1304), tensor(0.3073))

We want the mean and std to be as close as possible to 0 and 1 respectively.

In [6]:
x_train = normalize(x_train, train_mean, train_std)
# NB: Use training, not validation mean for validation set
x_valid = normalize(x_valid, train_mean, train_std)

In [7]:
train_mean, train_std = x_train.mean(), x_train.std()
train_mean, train_std

(tensor(-7.6999e-06), tensor(1.))

In [8]:
#export
def test_near_zero(a,tol=1e-3): assert a.abs()<tol, f"Near zero: {a}"

In [9]:
test_near_zero(train_mean)
test_near_zero(train_std-1)

In [10]:
n, m = x_train.shape
c = y_train.max()
n,m,c

(50000, 784, tensor(9))

## Foundations version

### Basic architecture

In [11]:
# num hidden
nh = 50

In [12]:
#simplified kaiming init/he init
w1 = torch.randn(m,nh)/math.sqrt(m)
b1 = torch.randn(nh)
w2 = torch.randn(nh,1)/math.sqrt(nh)
b2 = torch.randn(1)

In [13]:
w1.std()

tensor(0.0356)

In [14]:
test_near_zero(w1.mean())
test_near_zero(w1.std()-1/math.sqrt(m))

In [15]:
# This should be ~ (0,1) (mean,std)...
x_valid.mean(),x_valid.std()

(tensor(-0.0059), tensor(0.9924))

In [16]:
def lin(x, w, b): return x@w + b

In [17]:
t = lin(x_valid, w1, b1)

In [18]:
#...so should this, because we used kaiming init, which is designed to do this
t.mean(),t.std()

(tensor(-0.0159), tensor(1.4171))

We will be studying this in a lot of depth as to why this kaiming initalization was 'THE THING THAT MATTERS'

In [19]:
def relu(x): return x.clamp_min(0.) # clamp_min is basically max(0,x)

In [20]:
t = relu(lin(x_valid, w1, b1))

In [21]:
#...actually it really should be this!
t.mean(),t.std()

(tensor(0.5527), tensor(0.8353))

Kaiming he pointed out that the xavier init does not account for the impact of relu activation. This is a BIG problem. if your varaince halves each layer & you have a massive deep network then by the end your gradients will have totally dissapeared which is totally unacceptable. So they do something super genius which is they replace the 1 at the top with a 2.  
So we do the same thing.

From pytorch docs: `a: the negative slope of the rectifier used after this layer (0 for ReLU by default)`

$$\text{std} = \sqrt{\frac{2}{(1 + a^2) \times \text{fan_in}}}$$

This was introduced in the paper that described the Imagenet-winning approach from *He et al*: [Delving Deep into Rectifiers](https://arxiv.org/abs/1502.01852), which was also the first paper that claimed "super-human performance" on Imagenet (and, most importantly, it introduced resnets!)

In [22]:
# kaiming init/he init for relu
w1 = torch.randn(m,nh)*math.sqrt(2/m)

In [23]:
w1.mean(), w1.std()

(tensor(0.0004), tensor(0.0505))

In [24]:
t = relu(lin(x_valid, w1, b1))
t.mean(), t.std()

(tensor(0.6053), tensor(0.9495))

The result is now much closer. Its not perfect and it varies some but it is much better than before.

In [25]:
# export
from torch.nn import init

In [26]:
w1 = torch.zeros(m,nh)
init.kaiming_normal_(w1, mode='fan_out')#check docs for mode
t = relu(lin(x_valid, w1, b1))

In [27]:
#init.kaiming_normal_??
# Choosing ``'fan_in' preserves the magnitude of the variance of the weights in the
# forward pass. Choosing 'fan_out' preserves the magnitudes in the backwards pass.

In [28]:
w1.mean(), w1.std()

(tensor(-8.7934e-05), tensor(0.0505))

In [29]:
t.mean(), t.std()

(tensor(0.6188), tensor(0.9679))

In [30]:
w1.shape

torch.Size([784, 50])

In [31]:
import torch.nn
torch.nn.Linear(m,nh).weight.shape

torch.Size([50, 784])

In [32]:
torch.nn.Linear.forward??

In [33]:
torch.nn.functional.linear??

Turns out pytorch does a matrix product with a transpose.
So thats why we have to give the opposite information(fan-out) when we were trying to do it with our linear layer which doesn't have a transpose. This was just meant as to how to look into the pytorch soure code.

In [34]:
torch.nn.Conv2d??

In [35]:
torch.nn.modules.conv._ConvNd??

In [36]:
# what if..?
def relu(x): return x.clamp_min(0.) - 0.5

In [37]:
# kaiming init / he init for relu
w1 = torch.randn(m,nh)*math.sqrt(2./m )
t1 = relu(lin(x_valid, w1, b1))
t1.mean(),t1.std()

(tensor(0.2189), tensor(1.0031))

Defining a new version of relu got our mean very close to zero. It also helped a bit to the std(closer to 1 than before)

In [38]:
def model(xb):
    l1 = lin(xb, w1, b1)
    l2 = relu(l1)
    l3 = lin(l2, w2, b2)
    return l3

In [39]:
%timeit -n 10 _ = model(x_valid)

14.8 ms ± 1.86 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [40]:
torch.Size([x_valid.shape[0], 1])

torch.Size([10000, 1])

In [41]:
assert model(x_valid).shape==torch.Size([x_valid.shape[0],1])

### Loss function: MSE

In [42]:
model(x_valid).shape

torch.Size([10000, 1])

We need `squeeze()` to get rid of that trailing (,1), in order to use `mse`. (Of course, `mse` is not a suitable loss function for multi-class classification; we'll use a better loss function soon. We'll use `mse` for now to keep things simple.)  
output.squeeze() gets rid of all ones in your matrix but suppose if you have a btach size of 1 then it will get rid of too causing an error. Hence, its better to specify which dimension you wanna squeeze

In [43]:
#export
def mse(output, targ): return (output.squeeze(-1) - targ).pow(2).mean()

In [44]:
y_train, y_valid = y_train.float(), y_valid.float()

In [45]:
preds = model(x_train)
preds.shape

torch.Size([50000, 1])

In [46]:
mse(preds, y_train)

tensor(19.5981)

### Gradients and backward pass  


y^ = mse(lin2(relu(lin1(x))), y)  
DOING THE CHAIN RULE. WE'RE DOING EACH BIT SEPERATELY  

In [47]:
a = torch.zeros(3,3,1); a.shape

torch.Size([3, 3, 1])

In [48]:
a.squeeze().shape

torch.Size([3, 3])

In [49]:
a.unsqueeze(0).shape, a.unsqueeze(1).shape

(torch.Size([1, 3, 3, 1]), torch.Size([3, 1, 3, 1]))

In [53]:
def mse_grad(inp, targ):
    # grad of loss wrt the output of previous layer.
    # we store the gradient(derivative) in .g of the prev layer(which is inp)
    # so that input of mse(inp(parameter of this func)) is same as output of the previous layer(which is a Linear layer)
    inp.g = 2* (inp.squeeze()- targ).unsqueeze(-1)/inp.shape[0]

In [54]:
def relu_grad(inp, out):
    # grad of relu with respect to input activations
    # relu is max(0, x) so its deviative wil be:
    # 0 if x<0
    # 1 if x>0
    # since we're doing chain rule we gotta multiply it with the gradient of the next layer(which is inp.g in mse_grad)
    inp.g = (inp>0).float() * out.g

In [55]:
def lin_grad(inp, out, w, b):
    # grad of matmul wrt the input is simply the matrix product with the transpose.
    # we don't just need the gradient of the o/ps wrt the i/ps
    # but also wrt the weights and the biases. That's why we have three .g's here    
    inp.g = out.g @ w.t()###
    w.g = (inp.unsqueeze(-1)* out.g.unsqueeze(1)).sum(0)
    b.g = out.g.sum(0)

In [56]:
def forward_and_backward(inp, targ):
    # forward pass
    l1 = inp @ w1 + b1
    l2 = relu(l1)
    out = l2 @ w2 + b2
    # we don't actually need the loss in the backward pass
    loss = mse(out, targ)
    
    # backward pass
    mse_grad(out, targ)
    lin_grad(l2, out, w2, b2)
    relu_grad(l1, l2)
    lin_grad(inp, l1, w1, b1)    

In [57]:
%time forward_and_backward(x_train, y_train)

Wall time: 5.89 s


In [58]:
# saving for testing against the pytorch library
w1g = w1.g.clone()
w2g = w2.g.clone()
b1g = b1.g.clone()
b2g = b2.g.clone()
ig = x_train.g.clone()

We cheat a little bit and use PyTorch autograd to check our results.  
Pytorch stores the same thing in .grad instead of .g

In [62]:
xt2 = x_train.clone().requires_grad_(True)
w12 = w1.clone().requires_grad_(True)
w22 = w2.clone().requires_grad_(True)
b12 = b1.clone().requires_grad_(True)
b22 = b2.clone().requires_grad_(True)

In [63]:
def forward(inp, targ):
    # forward pass
    l1 = inp @ w12 + b12
    l2 = relu(l1)
    out = l2 @ w22 + b22
    # we don't actually need the loss in backward
    return mse(out, targ)

In [64]:
loss = forward(xt2, y_train)

In [65]:
loss.backward()

Testing if our calculated gradients are same to the ones tat were caluclated by Pytorch

In [66]:
test_near(w22.grad, w2g)
test_near(b22.grad, b2g)
test_near(w12.grad, w1g)
test_near(b12.grad, b1g)
test_near(xt2.grad, ig)

## Refactor model

### Layers as classes

In [68]:
class Relu():
    # if a class has a __call__ then we can treat it as a function.
    def __call__(self, inp):
        self.inp = inp
        self.out = inp.clamp_min(0.)-0.5
        return self.out
    
    def backward(self): self.inp.g = (self.inp>0).float() *self.out.g

In [69]:
class Lin():
    def __init__(self, w, b): 
        self.w, self.b = w, b
        
    def __call__(self, inp):
        self.inp = inp
        self.out = inp @ self.w + self.b
        return self.out
    
    def backward(self): 
        self.inp.g = self.out.g @ self.w.t()
        # creating a giant outer product just to sum it up is inefficient.
        self.w.g = (self.inp.unsqueeze(-1) * self.out.g.unsqueeze(1)).sum(0)
        self.b.g = self.out.g.sum(0)

In [70]:
class Mse():
    def __call__(self, inp, targ):
        self.inp = inp
        self.targ = targ
        self.out = (inp.squeeze() - targ).pow(2).mean()
    
    def backward(self):
        self.inp.g = 2 * (self.inp.squeeze()- self.targ).unsqueeze(-1)/ self.targ.shape[0]

In [71]:
class Model():
    def __init__(self, w1, b1, w2,  b2):
        self.layers = [Lin(w1, b1), Relu(), Lin(w2, b2)]
        self.loss = Mse()
        
    def __call__(self, x, targ):
        for l in self.layers: x = l(x)
        return self.loss(x, targ)
    
    def backward(self):
        self.loss.backward()
        for l in reversed(self.layers): l.backward()

In [72]:
w1.g, b1.g, w2.g, b2.g = [None]*4 # setting our gradients to None
model = Model(w1, b1, w2, b2)

In [73]:
%time loss = model(x_train, y_train)

Wall time: 72.7 ms


In [74]:
%time model.backward()

Wall time: 11.5 s


In [75]:
test_near(w2g, w2.g)
test_near(b2g, b2.g)
test_near(w1g, w1.g)
test_near(b1g, b1.g)
test_near(ig, x_train.g)

In [76]:
w1g.shape, w1.g.shape, w2g.shape, w2.g.shape

(torch.Size([784, 50]),
 torch.Size([784, 50]),
 torch.Size([50, 1]),
 torch.Size([50, 1]))

Refactoring it again!  
### Module.forward()

In [118]:
class Module():
    def __call__(self, *args):
        self.args = args
        self.out = self.forward(*args)
        return self.out
    
    def forward(self): raise Exception('not implemented')
    def backward(self): self.bwd(self.out, *self.args)

In [119]:
class Relu(Module):
    def forward(self, inp): return inp.clamp_min(0.)- 0.5
    def bwd(self, out, inp): inp.g = (inp>0).float() * out.g

In [139]:
class Lin(Module):
    def __init__(self, w, b): self.w, self.b = w, b
    
    def forward(self, inp): return inp @ self.w + self.b
    
    def bwd(self, out, inp): 
        inp.g = out.g @ self.w.t()
        self.w.g = torch.einsum('ai,aj->ij', inp, out.g)
        self.b.g = out.g.sum(0)

In [140]:
class Mse(Module):
    def forward (self, inp, targ): return (inp.squeeze() - targ).pow(2).mean()
    def bwd(self, out, inp, targ): inp.g = 2*(inp.squeeze()-targ).unsqueeze(-1) / targ.shape[0]    

In [141]:
class Model():
    def __init__(self):
        self.layers = [Lin(w1, b1), Relu(), Lin(w2, b2)]
        self.loss = Mse()
    
    def __call__(self, x, targ):
        for l in self.layers: x = l(x)
        return self.loss(x, targ)
    
    def backward(self):
        self.loss.backward()
        for l in reversed(self.layers): l.backward()

In [142]:
w1.g,b1.g,w2.g,b2.g = [None]*4
model = Model()

In [143]:
%time loss = model(x_train, y_train)

Wall time: 87.1 ms


In [144]:
%time model.backward()

Wall time: 234 ms


In [145]:
test_near(w2g, w2.g)
test_near(b2g, b2.g)
test_near(w1g, w1.g)
test_near(b1g, b1.g)
test_near(ig, x_train.g)

### Without einsum

In [146]:
class Lin(Module):
    def __init__(self, w, b): self.w,self.b = w,b
        
    def forward(self, inp): return inp@self.w + self.b
    
    def bwd(self, out, inp):
        inp.g = out.g @ self.w.t()
        self.w.g = inp.t() @ out.g
        self.b.g = out.g.sum(0)

In [147]:
w1.g,b1.g,w2.g,b2.g = [None]*4
model = Model()

In [148]:
%time loss = model(x_train, y_train)

Wall time: 77.5 ms


In [149]:
%time model.backward()

Wall time: 215 ms


In [150]:
test_near(w2g, w2.g)
test_near(b2g, b2.g)
test_near(w1g, w1.g)
test_near(b1g, b1.g)
test_near(ig, x_train.g)

### nn.Linear and nn.Module

In [151]:
# export
from torch import nn

In [154]:
class Model(nn.Module):
    def __init__(self, n_in, nh, n_out):
        super().__init__()
        self.layers = [nn.Linear(n_in, nh), nn.ReLU(), nn.Linear(nh, n_out)]
        self.loss = mse
        
    def __call__(self, x, targ):
        for l in self.layers: x = l(x)
        return self.loss(x, targ)
    

In [156]:
m, nh

(784, 50)

In [155]:
model = Model(m, nh, 1)

In [157]:
%time loss = model(x_train, y_train)

Wall time: 131 ms


In [158]:
%time loss.backward()

Wall time: 101 ms


Our forward pass is as fast as the one of Pytorch's but their backward pass is twice as fast than ours. Jeremy guess that the reason for that must be that they are only calculating the gradients that they need whereas we're calculating all of them.

## Export

In [159]:
!python notebook2script.py 02_fully_connected.ipynb

Converted 02_fully_connected.ipynb to exp\nb_02.py
